# **<div align="center">ADVANCE FEATURE ENGINEERING FOR BINANCE COIN </div>**

In [5]:
import pandas as pd
import numpy as np

In [6]:
def compute_RSI(series:pd.Serie, window:int = 14) -> pd.Serie:
    
    '''
    Calculate the Relative Strenght Index (RSI) for a price series in a temporal windows.

    Parameters:
        - series: price temporal series
        - wondows: the windows size. Default is 14 days, which is typical for this indicator 

    Returns:
        - rsi: Relative Strength Index series
    '''
    # Calculate price differences between consecutive periods
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    # Calculate rolling averages of gains and losses over the window
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()

    #Calculate RS (Relative Strengh)
    rs = avg_gain / avg_loss

    # Calculate RSI
    rsi = 100 - (100 / (1 + rs))

    return rsi

AttributeError: module 'pandas' has no attribute 'Serie'

In [4]:
def compute_MACD(series: pd.Series, fast: int = 12, slow: int = 26, 
                 signal: int = 9) -> tuple[pd.Series, pd.Series, pd.Series]:

    '''
    Calculate the Moving Average Convergence Divergence (MACD), between two rolling exponential averages (EMA)
    from two different periods

    Parameters:
        - series: price temporal series
        - fast: period of the fastest EMA (more sensitive). Detect recent changes. By default 12
        - slow: period of the slowest EMA (less sensitive). Detect genral tendency. By default 26
        - signal: period of the EMA of signal line to smooth MACD

    Returns:
        - macd: MACD line 
        - signal_line: EMA of the MACD line
        - hist: MACD histogram
    '''
    
    # Calculate the fast and slow exponential moving averages (EMA)
    ema_fast = series.ewm(span=fast, adjust=False).mean()
    ema_slow = series.ewm(span=slow, adjust=False).mean()

    # Calculate MACD line
    macd = ema_fast - ema_slow

    # Calculate signal line as EMA of the MACD line
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    
    # Calculate histogram
    hist = macd - signal_line

    return macd, signal_line, hist

NameError: name 'pd' is not defined

In [3]:
def compute_Bollinger_Bands(series, window=20, n_std=2):
    """Bollinger Bands"""
    sma = series.rolling(window=window).mean()
    std = series.rolling(window=window).std()
    upper = sma + n_std * std
    lower = sma - n_std * std
    return upper, lower

In [ ]:
def make_features(df: pd.DataFrame, price_col='price_usd', volume_col='volume', market_cap_col='market_cap'):
    """
    Crea features avanzados para series temporales de criptomonedas.
    """
    df = df.copy()
    df = df.sort_index()  # asegurarse de orden temporal
    
    # ========================
    # 1️⃣ Lags y retornos
    # ========================
    for lag in [1,2,3,7,14]:
        df[f'{price_col}_lag{lag}'] = df[price_col].shift(lag)
        df[f'{volume_col}_lag{lag}'] = df[volume_col].shift(lag)
        df[f'{market_cap_col}_lag{lag}'] = df[market_cap_col].shift(lag)
    
    # Retornos (ya lo tenías, pero recalculamos por seguridad)
    df['returns'] = df[price_col].pct_change() * 100
    df['volatility_7d'] = df['returns'].rolling(7).std()
    df['volatility_14d'] = df['returns'].rolling(14).std()
    df['returns_rolling_mean_7d'] = df['returns'].rolling(7).mean()
    
    # ========================
    # 2️⃣ Rolling windows
    # ========================
    for window in [7,14,30]:
        df[f'{price_col}_sma{window}'] = df[price_col].rolling(window).mean()
        df[f'{price_col}_std{window}'] = df[price_col].rolling(window).std()
        df[f'{volume_col}_sma{window}'] = df[volume_col].rolling(window).mean()
        df[f'{market_cap_col}_sma{window}'] = df[market_cap_col].rolling(window).mean()
    
    # ========================
    # 3️⃣ Indicadores técnicos
    # ========================
    df['RSI_14'] = compute_RSI(df[price_col], window=14)
    macd, signal_line, hist = compute_MACD(df[price_col])
    df['MACD'] = macd
    df['MACD_signal'] = signal_line
    df['MACD_hist'] = hist
    df['Bollinger_upper'], df['Bollinger_lower'] = compute_Bollinger_Bands(df[price_col])
    
    # ========================
    # 4️⃣ Features de calendario
    # ========================
    df['day_of_week'] = df.index.dayofweek  # lunes=0, domingo=6
    df['month'] = df.index.month
    df['quarter'] = df.index.quarter
    
    # ========================
    # 5️⃣ Limpieza de NAs por shifts y rolling
    # ========================
    df = df.dropna().reset_index(drop=False)
    
    return df
